# Lab 7: Topic Modeling

---
## 1. Notebook Overview

### 1.1 Objective
Build and interpret topic models on the tweet dataset using Gensim’s LDA implementation, then visualize and evaluate the discovered topics.

### 1.2 Prerequisites
This notebook assumes you have already executed:
- **Lab 2**: Data preprocessing → `../Data/tweets_preprocessed_train.parquet`, `../Data/tweets_preprocessed_test.parquet`, `../Data/tweets_preprocessed_validation.parquet`
- **Lab 3**: Language modeling
- **Lab 4**: Feature extraction → `../Data/top_1000_vocabulary.json`
- **Lab 5**: Neural network classification pipeline 
- **Lab 5 & 6**: Understanding of the dataset labels to compare topics with classes

### 1.3 Resources
- Gensim LDA documentation: https://radimrehurek.com/gensim/models/ldamodel.html
- pyLDAvis usage guide: https://pyldavis.readthedocs.io/en/latest/

### 1.4 Task Summary
- Train one or more LDA topic models with Gensim (consider preprocessing choices and reusing prior pipelines).
- Visualize the learned topics via pyLDAvis (`pyLDAvis.gensim_models.prepare(...)`).
- Interpret and critique the topics: which terms describe them? How do they align (or not) with the tweet label taxonomy?
- Experiment with multiple topic counts to see how granularity affects interpretability.

### 1.5 Section Roadmap
1. Section 2: Environment setup, data access, and preprocessing recap
2. Section 3: Build and train the Gensim LDA model
3. Section 4: Visualize topics with pyLDAvis
4. Section 5: Interpret topics, compare topic counts, and discuss insights

---
## 2. Data Loading & Preparation

### 2.1 Libraries
- `gensim` for LDA (`LdaModel`, `Dictionary`, `corpora`)
- `pyLDAvis.gensim_models` for interactive topic visualization
- `pandas`, `numpy`, `datasets` (HuggingFace) for loading the tweet splits

### 2.2 Installation Steps
1. Install/confirm `gensim`, `pyLDAvis`, and any preprocessing dependencies in the environment.
2. Ensure the preprocessed tweet file `../Data/tweets_preprocessed_train.parquet` and HuggingFace API access (`cardiffnlp/tweet_topic_multi`) are available.


In [1]:
%pip install --quiet gensim pyLDAvis numpy pandas 

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import warnings
import pandas as pd
import numpy as np
import typing
import json
import re
import pyLDAvis
import pyLDAvis.gensim_models


import gensim
from pathlib import Path
from typing import Sequence, Union, List
from datasets import load_dataset
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel, CoherenceModel

warnings.filterwarnings("ignore")

TRAIN_DATA_PATH = "../Data/tweets_preprocessed_train.parquet"
TEST_DATA_PATH = "../Data/tweets_preprocessed_test.parquet"
VAL_DATA_PATH = "../Data/tweets_preprocessed_validation.parquet"
VOCABULARY_PATH = "../Data/top_1000_vocabulary.json"
RANDOM_STATE = 42

In [3]:
# Load the Top 1000 vocabulary from Lab 4
print("Loading vocabulary from Lab 4...")
with open(VOCABULARY_PATH, 'r', encoding='utf-8') as f:
    vocab_data = json.load(f)

TOP_VOCABULARY = vocab_data['tokens']

print(f"✓ Loaded vocabulary from: {VOCABULARY_PATH}")
print(f"✓ Vocabulary size: {len(TOP_VOCABULARY)}")
print(f"✓ First 10 tokens: {TOP_VOCABULARY[:10]}")
print(f"✓ Description: {vocab_data. get('description', 'N/A')}")

Loading vocabulary from Lab 4...
✓ Loaded vocabulary from: ../Data/top_1000_vocabulary.json
✓ Vocabulary size: 1000
✓ First 10 tokens: ['new', 'day', 'love', 'good', 'game', 'year', 'time', 'watch', 'happy', 'music']
✓ Description: Top 1000 most frequent tokens from preprocessed tweets (Lab 4)


In [4]:
# Load preprocessed training data from Lab 2
print("\nLoading preprocessed training data from Lab 2...")
df_train = pd.read_parquet(TRAIN_DATA_PATH)
print(f"Training samples: {len(df_train):,}")
print(f"Columns: {df_train.columns.tolist()}")
df_train.head(3)


Loading preprocessed training data from Lab 2...
Training samples: 6,090
Columns: ['text', 'label_name', 'label']


,text,label_name,label
0,lumber beat rapid game western division final ...,['sports'],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,hear eli gold announce auburn game dumbass,['sports'],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,phone away try look home game ticket october,['sports'],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [5]:
print("Loading preprocessed test and validation data...")
df_test = pd.read_parquet(TEST_DATA_PATH)
df_val = pd.read_parquet(VAL_DATA_PATH)
print(f"Test samples: {len(df_test):,}")
print(f"Validation samples: {len(df_val):,}")

Loading preprocessed test and validation data...
Test samples: 1,679
Validation samples: 188


---
## 3. Train LDA Topic Models

### 3.1 Dictionary and Corpus
Tweets are re-tokenized with the Lab 2 preprocessing steps and restricted to the Lab 4 vocabulary so LDA concentrates on the most meaningful tokens. The `Dictionary` is built directly from these filtered tokens, preserving the entire vocabulary, including rare terms for the later interpretation work.

### 3.2 Topic Search Grid
To satisfy the requirements we sweep over a `topic_grid`. When the validation split retains enough tweets after vocabulary filtering we prioritize the `CoherenceModel (c_v)` score; otherwise we fall back to validation perplexity plus manual inspection because the validation subset is relatively small.


In [6]:
def ensure_tokens(sentence: Union[Sequence[str], str]) -> List[str]:
    """Whitespace tokenizer reused from previous labs."""
    if isinstance(sentence, str):
        sentence = sentence.split()
    return list(sentence)

def tokenize_column(series, vocabulary=None):
    """Tokenize tweets and optionally restrict tokens to a curated vocabulary."""
    docs, kept_idx = [], []
    for idx, row in enumerate(series):
        tokens = ensure_tokens(row)
        if vocabulary is not None:
            tokens = [tok for tok in tokens if tok in vocabulary]
        if tokens:
            docs.append(tokens)
            kept_idx.append(idx)
    return docs, kept_idx

def normalize_label(entry):
    """Reduce multi-hot or string labels to a single representative tag."""
    if isinstance(entry, list) and entry:
        return entry[0]
    if isinstance(entry, str):
        matches = re.findall(r"[A-Za-z0-9_&]+", entry)
        if matches:
            return matches[0]
        return entry.strip()
    return "unknown"

In [ ]:
# Restrict documents to the curated Lab 4 vocabulary for cleaner topics.
top_vocab = set(TOP_VOCABULARY)

train_df = pd.read_parquet(TRAIN_DATA_PATH)
val_df = pd.read_parquet(VAL_DATA_PATH)

train_tokens, train_idx = tokenize_column(train_df["text"], vocabulary=top_vocab)
val_tokens, val_idx = tokenize_column(val_df["text"], vocabulary=top_vocab)

# Align labels with the filtered token lists so later evaluation uses matching rows.
train_labels = [normalize_label(train_df["label_name"].iloc[i]) for i in train_idx]
val_labels = [normalize_label(val_df["label_name"].iloc[i]) for i in val_idx]

print(f"Docs kept → train: {len(train_tokens)}, val: {len(val_tokens)}")
print(f"Sample tokens: {train_tokens[0][:15]}")



Docs kept → train: 6084, val: 188
Sample tokens: ['beat', 'game', 'final', 'hit']


In [8]:
dictionary = gensim.corpora.Dictionary(train_tokens)

train_corpus = [dictionary.doc2bow(text) for text in train_tokens]
val_corpus = [dictionary.doc2bow(text) for text in val_tokens]

print(f"Dictionary size: {len(dictionary)}")
print(f"Train documents: {len(train_corpus)}, Validation documents: {len(val_corpus)}")


Dictionary size: 1000
Train documents: 6084, Validation documents: 188


In [14]:
def train_lda(num_topics, passes=12):
    return LdaModel(
        corpus=train_corpus,
        num_topics=num_topics,
        id2word=dictionary,
        random_state=RANDOM_STATE,
        chunksize=2048,
        passes=passes,
        alpha="auto",
        eta="auto",
        minimum_probability=0.0,
    )

topic_grid = [15, 19, 35, 45] # label-aligned, mid, and high-granularity k
lda_runs, rows = {}, []

for k in topic_grid:
    model = train_lda(k)
    lda_runs[k] = model

    coherence_model = CoherenceModel(
        model=model,
        texts=train_tokens, # val tokens too sparse for stable c_v
        dictionary=dictionary,
        coherence="c_v",
    )
    coherence = coherence_model.get_coherence()
    perplexity = model.log_perplexity(val_corpus)

    rows.append(
        {
            "topics": k,
            "coherence_c_v": coherence,
            "val_perplexity": perplexity,
        }
    )
    print(f"k={k:2d} → coherence={coherence:.3f}, perplexity={perplexity:.3f}")




k=15 → coherence=0.321, perplexity=-13.834
k=19 → coherence=0.337, perplexity=-14.560
k=35 → coherence=0.420, perplexity=-14.998
k=45 → coherence=0.445, perplexity=-15.297


### Topic sweep (k = 19, 35, 45) with coherence + perplexity
- 19 topics mirror the 19 supervised labels in the dataset, so we can compare them directly.  
- 35 topics act as a mid-granularity level that already splits subthemes.  
- 45 topics gave the best trade-off between perplexity and interpretable coherence in earlier runs.  
We compute coherence on `train_tokens` because the validation split shrinks too much after the vocabulary filter and would otherwise return NaN; the test set stays untouched.

In [ ]:
metrics_df = (
    pd.DataFrame(rows)
    .sort_values(["coherence_c_v", "val_perplexity"], ascending=[False, True])
    .reset_index(drop=True)
)
display(metrics_df)

best_topic_count = int(metrics_df.iloc[0]["topics"])
best_model = lda_runs[best_topic_count]
print(f"Chosen topic count: {best_topic_count}")

,topics,coherence_c_v,val_perplexity
0,45,0.445071,-15.296798
1,35,0.419684,-14.997864
2,19,0.336687,-14.560079
3,15,0.321070,-13.834461


Chosen topic count: 45


---
## 4. Visualize and Inspect Topics

### 4.1 Interactive pyLDAvis
Use `pyLDAvis.gensim.prepare(...)` to render the intertopic distance map (bubble plot) and the top-term bar chart for any trained model. Bubble size encodes how prevalent a topic is in the corpus, distances highlight similarity, and the λ slider controls whether the term list emphasizes raw frequency (λ ≈ 1) or exclusivity (λ ≈ 0). Exported HTML dashboards saved in `../Data/pyldavis_k19.html` and `../Data/pyldavis_k45.html`.

### 4.2 Tabular Keyword Summary
Because notebooks are often shared as static reports, a compact table of the top-N keywords per topic is generated alongside the visualizations. It’s handy for quick scans (e.g., pasting into slides) or when pyLDAvis can’t be opened.


In [19]:
print("Preparing pyLDAvis visualization for k=19 topics ...")
panel_k19 = pyLDAvis.gensim.prepare(
    lda_runs[19],
    train_corpus,
    dictionary,
    sort_topics=False,
)
pyLDAvis.enable_notebook()
pyLDAvis.display(panel_k19)
panel_k19

Preparing pyLDAvis visualization for k=19 topics ...


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0     -0.015213  0.087424       1        1  4.420365
1      0.193041  0.031087       2        1  4.396888
2      0.128414 -0.096700       3        1  4.995478
3      0.049825  0.216234       4        1  5.022940
4     -0.152126  0.087179       5        1  5.310978
5     -0.156631 -0.129062       6        1  6.292764
6      0.077893  0.014587       7        1  6.313810
7      0.193251 -0.114150       8        1  4.690834
8      0.140142  0.123977       9        1  4.432237
9     -0.047716 -0.029826      10        1  5.581213
10    -0.140825 -0.120942      11        1  6.130437
11    -0.002896 -0.158942      12        1  5.361151
12    -0.007299  0.005668      13        1  4.653561
13     0.060268 -0.011148      14        1  5.908877
14    -0.178624 -0.040829      15        1  5.662251
15    -0.153863  0.174256      16        1  6.214002
16    -0.049275  0.163302      17        1  4.855139
17     0.131020 -0.055346      18        1  4.896144
18    -0.069386 -0.146769      19        1  4.860929, topic_info=       Term        Freq       Total Category  logprob  loglift
47      new  571.000000  571.000000  Default  30.0000  30.0000
420   music  361.000000  361.000000  Default  29.0000  29.0000
295    love  532.000000  532.000000  Default  28.0000  28.0000
55    video  293.000000  293.000000  Default  27.0000  27.0000
21      win  302.000000  302.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
39    watch   42.838632  376.888008  Topic19  -4.0400   0.8494
18    thank   39.306096  308.328305  Topic19  -4.1261   0.9642
54     good   30.159602  446.792635  Topic19  -4.3910   0.3283
62   stream   25.945018  120.825762  Topic19  -4.5415   1.4856
81     play   25.261751  282.530501  Topic19  -4.5682   0.6094

[913 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
961       6  0.927876       aaron
165       1  0.882808        able
165      13  0.101862        able
675       1  0.189467  absolutely
675       7  0.031578  absolutely
...     ...       ...         ...
514      16  0.278019       young
514      17  0.162178       young
496       5  0.166217     youtube
496       8  0.055406     youtube
496      15  0.720272     youtube

[2049 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])

In [20]:
print("Preparing pyLDAvis visualization for k=45 topics ...")
panel_k45 = pyLDAvis.gensim.prepare(
    lda_runs[45],
    train_corpus,
    dictionary,
    sort_topics=False,
)
pyLDAvis.display(panel_k45)
panel_k45

Preparing pyLDAvis visualization for k=45 topics ...


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0     -0.026087  0.008202       1        1  2.770975
1      0.079591  0.013261       2        1  1.769094
2     -0.029582  0.105433       3        1  2.177838
3      0.013058  0.040722       4        1  1.462776
4     -0.018112  0.172476       5        1  1.970314
5      0.153994 -0.184231       6        1  2.633975
6     -0.073544 -0.096539       7        1  2.117185
7      0.092292  0.142211       8        1  2.162312
8     -0.034090 -0.036055       9        1  1.894605
9     -0.074407  0.026305      10        1  1.775512
10    -0.154530  0.093092      11        1  2.562440
11     0.030444  0.101458      12        1  2.062120
12    -0.119335  0.008550      13        1  1.925380
13    -0.058702 -0.048722      14        1  2.329100
14    -0.020556 -0.117852      15        1  2.007148
15     0.137497 -0.062396      16        1  2.130648
16    -0.069244 -0.117329      17        1  2.791990
17    -0.041337  0.122878      18        1  1.834457
18     0.104944  0.018999      19        1  2.264028
19    -0.047279  0.054746      20        1  2.265592
20     0.000159 -0.127382      21        1  2.354574
21    -0.016591 -0.101293      22        1  2.157666
22    -0.098758 -0.052538      23        1  2.738774
23    -0.081813  0.101909      24        1  1.639112
24    -0.061816 -0.112343      25        1  2.169913
25     0.101359  0.010269      26        1  2.310141
26     0.179312 -0.018771      27        1  3.161101
27     0.118645  0.082313      28        1  2.169996
28    -0.036347 -0.053222      29        1  2.414624
29     0.208222  0.037002      30        1  2.244174
30     0.026107 -0.118220      31        1  2.826379
31    -0.073110  0.087036      32        1  2.179261
32     0.234850  0.076136      33        1  2.413035
33     0.011375  0.006976      34        1  2.069044
34     0.169999  0.009437      35        1  1.967262
35    -0.073174  0.011626      36        1  2.350783
36    -0.058774  0.055679      37        1  2.100343
37    -0.040564  0.039691      38        1  2.022940
38    -0.051975  0.064196      39        1  2.082433
39    -0.072330  0.015689      40        1  1.791988
40    -0.061282 -0.099142      41        1  2.047867
41     0.079687 -0.135068      42        1  3.427336
42    -0.124898 -0.102979      43        1  2.243229
43    -0.043302  0.068385      44        1  2.695065
44    -0.079996  0.009407      45        1  1.515473, topic_info=        Term        Freq       Total Category  logprob  loglift
295     love  537.000000  537.000000  Default  30.0000  30.0000
169      day  504.000000  504.000000  Default  29.0000  29.0000
420    music  349.000000  349.000000  Default  28.0000  28.0000
22      year  408.000000  408.000000  Default  27.0000  27.0000
55     video  282.000000  282.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
83      time   19.652478  396.576736  Topic45  -3.6538   1.1848
39     watch   15.288323  380.603028  Topic45  -3.9049   0.9748
178   people   10.359990  224.694275  Topic45  -4.2940   1.1127
87     great   10.564777  303.375887  Topic45  -4.2744   0.8320
129  morning    8.713573  180.786202  Topic45  -4.4671   1.1570

[1620 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
961      43  0.923536       aaron
165       6  0.819091        able
165      21  0.170644        able
480      22  0.952985    absolute
675      15  0.947929  absolutely
...     ...       ...         ...
514      38  0.980815       young
496       3  0.055348     youtube
496      24  0.885569     youtube
219      11  0.381630        zone
219      18  0.572445        zone

[2088 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31

In [ ]:
# Save interactive pyLDAvis visualizations as html files
pyLDAvis.save_html(panel_k19, "../Data/pyldavis_k19.html")
pyLDAvis.save_html(panel_k45, "../Data/pyldavis_k45.html")

def summarize_topics(model: LdaModel, num_words: int = 12) -> pd.DataFrame:
    """Collect top-n keywords per topic for quick, non-interactive inspection."""
    rows = []
    for topic_id, word_info in model.show_topics(
        num_topics=-1, num_words=num_words, formatted=False
    ):
        # Join the per-topic (word, weight) tuples into a readable comma-separated string.
        tokens = ", ".join(word for word, _ in word_info)
        rows.append({"topic": topic_id, "keywords": tokens})
    return pd.DataFrame(rows).sort_values("topic")


print("Top keywords for k=19 model")
topics_k19 = summarize_topics(lda_runs[19], num_words=12)
display(topics_k19)

print("Top keywords for k=45 model")
topics_k45 = summarize_topics(lda_runs[45], num_words=12)
display(topics_k45)

Top keywords for k=19 model


,topic,keywords
0,0,"break, match, love, like, see, return, time, b..."
1,1,"stay, safe, harry, king, red, online, sugar, w..."
2,2,"change, power, climate, time, close, take, get..."
3,3,"day, sign, family, petition, national, light, ..."
4,4,"thank, bad, great, late, day, sta, movie, dail..."
5,5,"win, game, night, team, state, super, today, f..."
6,6,"new, music, video, song, album, official, year..."
7,7,"news, morning, pm, talk, ground, plan, york, f..."
8,8,"check, like, drop, come, add, leave, feel, let..."
9,9,"trump, vote, work, weekend, say, boy, presiden..."


Top keywords for k=45 model


,topic,keywords
0,0,"love, break, kid, bad, make, coach, de, kill, ..."
1,1,"harry, tweet, sugar, sell, watermelon, style, ..."
2,2,"say, change, climate, state, fact, learn, shar..."
3,3,"fan, expect, release, performance, eat, mind, ..."
4,4,"help, sta, drop, money, job, face, people, voi..."
5,5,"happy, friend, long, god, year, wish, lot, cel..."
6,6,"life, real, matter, black, fall, special, spea..."
7,7,"power, pm, save, spotify, group, government, p..."
8,8,"story, player, league, write, coronavirus, com..."
9,9,"sunday, country, baby, july, fun, didn, believ..."


---
## 5. Interpret Topics and Align Them with Dataset Labels

### 5.1 Topic → Label Alignment

Each tweet is assigned to the topic with the highest posterior probability and paired with its supervised label from the dataset. Aggregating these `(topic, label)` pairs lets us inspect, per topic, which labels dominate, how often a label appears, and how confident the model was when it picked that topic. The resulting table exposes:

- **Primary labels per topic** – high `count` + `topic_share` values show that, say, Topic 10 is mostly `sports`, while Topic 5 blends `diaries_&_daily_life` and `news_&_social_concern`.
- **Mixed topics** – topics whose top three labels all have similar shares indicate overlapping vocabulary or label definitions (useful for revisiting the taxonomy).
- **Confidence signals** – `avg_confidence` highlights whether the dominant label-topic pairing is strong (>0.4) or tentative (<0.25), guiding where further preprocessing or more topics might help.

In [23]:
def topic_label_alignment(model: LdaModel, corpus, labels, top_n: int = 3) -> pd.DataFrame:
    assignments = []
    for bow, label in zip(corpus, labels):
        topic_id, topic_prob = max(
            model.get_document_topics(bow, minimum_probability=0.0),
            key=lambda item: item[1],
        )
        assignments.append((topic_id, label, topic_prob))

    alignment = (
        pd.DataFrame(assignments, columns=["topic", "label", "probability"])
        .groupby(["topic", "label"])
        .agg(count=("probability", "size"),
             avg_confidence=("probability", "mean"))
        .reset_index()
    )
    alignment["topic_share"] = alignment.groupby("topic")["count"].transform(
        lambda counts: counts / counts.sum()
    )
    return alignment.sort_values(["topic", "count"], ascending=[True, False]) \
                    .groupby("topic").head(top_n)


print("Topic ↔ label alignment (k=19)")
alignment_k19 = topic_label_alignment(lda_runs[19], train_corpus, train_labels)
display(alignment_k19)

print("Topic ↔ label alignment (k=45)")
alignment_k45 = topic_label_alignment(lda_runs[45], train_corpus, train_labels)
display(alignment_k45)

Topic ↔ label alignment (k=19)


,topic,label,count,avg_confidence,topic_share
2,0,celebrity_&_pop_culture,52,0.427803,0.220339
11,0,news_&_social_concern,52,0.454671,0.220339
15,0,sports,41,0.461479,0.173729
19,1,celebrity_&_pop_culture,65,0.490125,0.271967
32,1,sports,64,0.544439,0.267782
18,1,business_&_entrepreneurs,19,0.564757,0.079498
45,2,news_&_social_concern,90,0.499813,0.307167
36,2,celebrity_&_pop_culture,54,0.465171,0.184300
49,2,sports,42,0.414267,0.143345
63,3,news_&_social_concern,69,0.459290,0.236301


Topic ↔ label alignment (k=45)


,topic,label,count,avg_confidence,topic_share
2,0,celebrity_&_pop_culture,46,0.271113,0.221154
6,0,film_tv_&_video,35,0.422010,0.168269
3,0,diaries_&_daily_life,24,0.288800,0.115385
18,1,celebrity_&_pop_culture,44,0.403313,0.483516
17,1,business_&_entrepreneurs,12,0.596196,0.131868
...,...,...,...,...,...
616,43,celebrity_&_pop_culture,31,0.288998,0.144186
629,43,sports,30,0.277053,0.139535
642,44,sports,24,0.335935,0.266667
640,44,news_&_social_concern,11,0.309892,0.122222


### 5.2 Discussion and Observations

- **k = 19 (label-aligned)**: Most latent topics map cleanly to a single supervised class: `sports`, `music`, and `news_&_social_concern` dominate individual topics with ~70–80 % share. This makes the model easy to explain to stakeholders, yet some clusters still mix politics with general news or lifestyle chatter, showing that 19 topics can’t perfectly isolate every class.
- **k = 45 (metric best)**: Coherence improves and perplexity drops. The pyLDAvis view shows tightly packed bubbles for climate activism, NBA trade rumors, celebrity obituaries, pandemic updates, etc. These subtopics are great for exploratory analysis but fragment the supervised classes (multiple sports/politics bubbles), so this model trades simplicity for detail.
- **Topic divergences**: Several topics capture conversational filler (“good morning”, “happy birthday”) or sustained climate discussions that have no dedicated label, suggesting blind spots in the taxonomy. Climate-related tweets drifting away from general news at k=45 hint that an “environment” class could be useful.

### 5.3 How the Topics Inform the Dataset

- **Alignment with classes**: When k equals the class count, topics mostly reflect the label definitions, confirming that the preprocessing from earlier labs preserved the label structure.
- **Where to use which model**: Use the k=19 model when you need consistency with the supervised classifier or want to annotate new tweets with the existing classes. Use the k=45 model for editorial research, content audits, or to surface emerging subthemes ahead of creating new labels.
- **Practical takeaway**: Topic modeling highlights both well-covered areas (sports, entertainment) and underrepresented concepts (climate, casual chatter). This helps prioritize where to extend the label inventory or build specialized classifiers.

### 5.4 Visual Exploration

Inspect the exported pyLDAvis dashboards to drill into word distributions and bubble overlaps:

```bash
open grundlagen-des-nlp-ws25_26/Abgabe/Data/pyldavis_k19.html
open grundlagen-des-nlp-ws25_26/Abgabe/Data/pyldavis_k45.html
```

